In [1]:
import numpy as np
import tensorly as tl
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
import torch
import torchvision

from torchsummary import summary

In [6]:
resnet18 = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/egor/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [12]:
resnet18.cuda()
summary(resnet18, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [28]:
for p in resnet18.named_parameters():
    print(p[0])

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

In [289]:
Y = resnet18.layer4[1].conv1.weight.data.detach().cpu().clone().numpy()
Y = Y.reshape(512,512,9).transpose([2,0,1])
Y = np.transpose(Y, [1,2,0])

In [290]:
Y.shape

(512, 512, 9)

In [282]:
R = 50
R1, R2 = 30,30

In [297]:
core.shape

(90, 90, 9)

In [302]:
U = np.diag(np.ones(90))[:,:R1]
V = np.diag(np.ones(90))[:,:R2]
A = np.random.randn(R1,R)
B = np.random.randn(R2,R)
C = np.random.randn(9,R)

#Y_restored = np.transpose(tl.kruskal_to_tensor((np.ones(R), (U@A, V@B, C))), [0,1,2])

#print("Random tensors. Norm is", np.linalg.norm(Y-Y_restored))

In [307]:
Y1 = tl.unfold(core,0)
for i in range(20):
    #print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
         # np.linalg.norm(Y1))
    A = U.T @ tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B])) @ np.linalg.inv(C.T@C * (B.T@B))
    Q = tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B]))@ np.linalg.inv((C.T@C * (B.T@B))) @\
        tl.tenalg.khatri_rao([C, V@B]).T @ tl.unfold(core,0).T

    eigvals, eigvecs = np.linalg.eigh(Q)
    indices = np.argsort(np.abs(eigvals))[-R1:]
    
    U = eigvecs[:,indices]
    
    B = V.T @ tl.unfold(core,1) @ (tl.tenalg.khatri_rao([C, U@A])) @ np.linalg.inv(C.T@C * (A.T@A))
    """
    R_ = tl.unfold(core,1) @ tl.tenalg.khatri_rao([C, U@A]) @ np.linalg.inv(C.T@C * (A.T@A)) @ \
        tl.tenalg.khatri_rao([C, U@A]).T@tl.unfold(core,1).T
    eigvals, eigvecs = np.linalg.eig(R_)
    indices = np.argsort(np.abs(eigvals))[-R2:]
    V = eigvecs[:,indices]
    
    C = tl.unfold(core,2) @ (tl.tenalg.khatri_rao([U@A, V@B])) @ np.linalg.inv(B.T@B * (A.T@A))
    #C[C < 0] = 0
    
    """
    
   

    
    
    print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
          np.linalg.norm(Y1))

Relative Norm is 18.190965891528155
Relative Norm is 55.72791856522463
Relative Norm is 36.56090282025632
Relative Norm is 33.93444784652989
Relative Norm is 42.87015802962677
Relative Norm is 227.06430247442577
Relative Norm is 38.463958262867614
Relative Norm is 75.43506778073713
Relative Norm is 54.737510400389006
Relative Norm is 60.43710728862648
Relative Norm is 27.575896838148743
Relative Norm is 46.812581269660406
Relative Norm is 37.49252651074015
Relative Norm is 14.538573546039027
Relative Norm is 106.04868299273694
Relative Norm is 19.562351070977858
Relative Norm is 25.264579092814405
Relative Norm is 35.41485944195529
Relative Norm is 33.11529809550182
Relative Norm is 2095.3395462344997


In [286]:
from tensorly.decomposition import partial_tucker

In [291]:
core, factors = partial_tucker(Y, 
                               modes=[0,1],
                               ranks=[90,90],
                               n_iter_max=200, 
                               tol=1e-6)

In [292]:
core.shape

(90, 90, 9)

In [293]:
core_weights, core_factors = tl.decomposition.parafac(core,
                         rank=150, 
                         n_iter_max=5000,
                         tol=1e-7)

In [294]:
restored_core = tl.kruskal_to_tensor((core_weights, core_factors))

In [296]:
np.linalg.norm(core-restored_core)/np.linalg.norm(core)

0.131139